# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from decimal import Decimal

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
db = mongo['uk_food']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food')

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# review the collections in our new database
collection_names = db.list_collection_names()
print("Collections in the 'uk_food' database:")
pprint(collection_names)

Collections in the 'uk_food' database:
['establishments']


In [7]:
# review a document in the establishments collection
establishments = db['establishments']

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
}

In [10]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)

In [11]:
# Check that the new restaurant was inserted
if result.acknowledged:
    print("Penang Flavours restaurant was successfully added.")
else:
    print("Failed to add Penang Flavours restaurant.")

Penang Flavours restaurant was successfully added.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}
business_type_projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

business_type_info = establishments.find_one(business_type_query, business_type_projection)
pprint(business_type_info)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID

update_result = establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {
        "$set": {
            "BusinessTypeID": 1
        }
    }
)

# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("Penang Flavours restaurant was successfully updated with BusinessTypeID.")
else:
    print("Failed to update Penang Flavours restaurant.")

Failed to update Penang Flavours restaurant.


In [14]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("Penang Flavours restaurant was successfully updated with BusinessTypeID.")
else:
    print("Failed to update Penang Flavours restaurant.")

Failed to update Penang Flavours restaurant.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
dover_count = establishments.count_documents(dover_query)
print(f"Number of establishments in Dover: {dover_count}")

Number of establishments in Dover: 0


In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many(dover_query)

In [17]:
# Check if any remaining documents include Dover
dover_count_after_delete = establishments.count_documents(dover_query)
if dover_count_after_delete == 0:
    print("All establishments in Dover have been deleted.")
else:
    print(f"Failed to delete all establishments in Dover. Remaining count: {dover_count_after_delete}")

All establishments in Dover have been deleted.


In [18]:
# Check that other documents remain with 'find_one'
# Check that the coordinates and rating value are now numbers
updated_document = establishments.find_one({"geocode.longitude": {"$type": 1}, "geocode.latitude": {"$type": 1}})
if updated_document:
    print("Sample document with updated data types:")
    pprint(updated_document)
else:
    print("No documents with updated data types found.")

No documents with updated data types found.


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many(
    {
        "geocode.longitude": {"$exists": True, "$type": "string"},
        "geocode.latitude": {"$exists": True, "$type": "string"}
    },
    [
        {
            '$set': {
                "geocode.longitude": {'$toDouble': "$geocode.longitude"},
                "geocode.latitude": {'$toDouble': "$geocode.latitude"}
            }
        }
    ]
)

Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [21]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [
    {
        '$set': {
            "RatingValue": {'$toInt': "$RatingValue"}
        }
    }
])

In [22]:
# Check that the coordinates and rating value are now numbers
sample_document = establishments.find_one()
pprint(sample_document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64fba053d2cd313c7f5428de'),
 'geocode': {'coordinates': [1.195625, 51.083812],
             'latitude': None,
             'longitude': None,
             'type': 'Point'},
 'links': [{'href': 'https://api.ratings.food.gov.uk